In [7]:
import boto3

In [6]:
AWS_ACCESS_KEY_ID = "your_access_key"
AWS_SECRET_ACCESS_KEY = "your_secret_key"
AWS_SESSION_TOKEN = "your_session_token"

In [31]:
bucket = "f14-datalake-landing-dev"
prefix = "scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin"

In [30]:
# Cliente S3
s3 = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
)


# Dump Fn from S3
def s3_items_from_table(bucket, prefix):
    paginator = s3.get_paginator("list_objects_v2")
    pages = paginator.paginate(Bucket=bucket, Prefix=prefix)
    objects = []
    for page in pages:
        for item in page["Contents"]:
            objects.append(item["Key"])

    return objects

In [32]:
# list S3 Objects
files = s3_items_from_table(bucket=bucket, prefix=prefix)
print(f"{len(files)} items in total")

4382 items in total


In [49]:
# Copy Fn
def copy_s3_object(bucket, key):
    copy_source = {"Bucket": bucket, "Key": key}
    s3.copy_object(
        CopySource=copy_source, Bucket=bucket, Key=key, MetadataDirective="REPLACE"
    )

In [33]:
files[:5]

['scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-17/p_brand_id=BAHV/p_region_id=NA/p_country_id=US/bahv-na-us-detailsalestrafficbychilditem-2019-11-17.csv',
 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-17/p_brand_id=BARVIVO/p_region_id=NA/p_country_id=US/barvivo-na-us-detailsalestrafficbychilditem-2019-11-17.csv',
 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-18/p_brand_id=BARVIVO/p_region_id=NA/p_country_id=US/barvivo-na-us-detailsalestrafficbychilditem-2019-11-18.csv',
 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-19/p_brand_id=BARVIVO/p_region_id=NA/p_country_id=US/barvivo-na-us-detailsalestrafficbychilditem-2019-11-19.csv',
 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-20/p_brand_id=BARVIVO/p_region_id=NA/p_country_id=

In [44]:
list_to_copy = []

for i in files:
    if "BARVIVO" not in i and "TRIBE_WOD" not in i:
        list_to_copy.append(i)

print(len(list_to_copy))

3344


In [45]:
list_to_copy[:5]

['scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-17/p_brand_id=BAHV/p_region_id=NA/p_country_id=US/bahv-na-us-detailsalestrafficbychilditem-2019-11-17.csv',
 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-21/p_brand_id=BAHV/p_region_id=NA/p_country_id=US/bahv-na-us-detailsalestrafficbychilditem-2019-11-21.csv',
 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-24/p_brand_id=MOUNTAIN_GRILLERS/p_region_id=NA/p_country_id=US/mountain_grillers-na-us-detailsalestrafficbychilditem-2019-11-24.csv',
 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-24/p_brand_id=SWEET_VIBES/p_region_id=NA/p_country_id=US/sweet_vibes-na-us-detailsalestrafficbychilditem-2019-11-24.csv',
 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-24/p_brand_id=WWO/p_region_i

In [51]:
# Copying elements one by one
if len(list_to_copy) > 0:
    c = 0
    for item in sorted(list_to_copy[:1]):
        try:
            copy_s3_object(bucket, item)
            # time.sleep(sleep_between_files)
            c += 1
            print(f"OK:   file_nro:{c}   >>{item}<<")
        except Exception as exception:
            print(f"FAILED:   >>{item}<<")
            print(f"FAILED REASON:   >>{exception}<<")

    print(f"Process Finished: {len(list_to_copy)} copied")
else:
    print("Nothing to move")

OK:   file_nro:1   >>scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2019-11-17/p_brand_id=BAHV/p_region_id=NA/p_country_id=US/bahv-na-us-detailsalestrafficbychilditem-2019-11-17.csv<<
Process Finished: 3344 copied
